In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import os
import shutil
directory_out = './file/2.url'
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import json

# = = =  = = =  = = =

directory_in = './file/1.item_number'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                           header=0,
                                                                           dtype=str).fillna('')
    print('Loading Done !')
    print()

    df_input['No'] = df_input['No'].astype(int)

    crawler_remain = len(df_input)

    print(f'总数量：{crawler_remain}')
    print()

    # = = =  = = =  = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, crawler_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = =  = = =  = = =

        try:
            crawler_retry = 0
            while True:
                crawler_retry += 1

                try:
                    url_request = f'''https://www.ebay.com/itm/{crawler_series['Item Number']}?_ul=US&_stpos=91710&orig_cvip=true'''

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 404:
                        break
                    elif resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        # = = =  = = =  = = =

                        dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

                        # = = =  = = =  = = =

                        if dict_['o']['g']['itemId'].strip() == crawler_series['Item Number']:
                            list_dict = [list_[2] for list_ in dict_['o']['w'] if 'model' in list_[2]]

                            # = = =  = = =  = = =

                            for link in list_dict[0]['model']['modules']['MARS']['metaData']:
                                soup = BeautifulSoup(link, 'lxml')
                                html = etree.HTML(str(soup))

                                # = = =  = = =  = = =

                                if html is not None and html.xpath('//link[@hreflang="x-default"]'):
                                    url = html.xpath('//link/@href')[0]

                                    if url.startswith('https://www.ebay.com/'):
                                        url = f'{url}?_ul=US&_stpos=91710&orig_cvip=true'
                                    elif url.startswith('https://www.ebay.de/'):
                                        url = f'{url}?_ul=DE&_stpos=10115&orig_cvip=true'
                                    elif url.startswith('https://www.ebay.co.uk/'):
                                        url = f'{url}?_ul=GB&_stpos=BS81QU&orig_cvip=true'
                                    elif url.startswith('https://www.ebay.com.au/'):
                                        url = f'{url}?_ul=AU&_stpos=2019&orig_cvip=true'
                                    elif url.startswith('https://www.ebay.ca/'):
                                        url = f'{url}?_ul=CA&_stpos=M5S2E8&orig_cvip=true'

                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = =  = = =  = = =

            if resp.status_code == 404:
                raise

            # = = =  = = =  = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Item Number': crawler_series['Item Number'],
                                          'Url': url})

            # = = =  = = =  = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = =  = = =  = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and crawler_remain == 1 or crawler_remain % 30:
            clear_output()
        print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            crawler_remain -= 1

    # = = =  = = =  = = =

    pool.map(crawler, df_input.iterrows())

    print('Data Outputting...')
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(os.path.join(directory_out, f'''{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
    print('Output Finished !')
    print()
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-url_error.xlsx''', index=False)
        print('- Error Occurred -')
        print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

[True] - [请求2次] - [剩余0条] - [crawler_2（100.00%）] - [2024/12/05 20:53:28] - 4484. https://www.ebay.com/itm/303992100477?_ul=US&_stpos=91710&orig_cvip=true

Data Outputting...
Output Finished !

Done ~
